# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

# For text preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print('Libraries imported successfully!')

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

# Load the dataset
df = pd.read_csv('indo_spam.csv')

# Display basic information
print('Dataset loaded successfully!')
print(f'Dataset shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
print('\nFirst 5 rows:')
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

Cek Info Dataset, duplikat data dan missing values

In [ ]:

print('=== Dataset Information ===')
df.info()
print('\n=== Statistical Summary ===')
print(df.describe())
print('\n=== Missing Values ===')
print(df.isnull().sum())
print(f'\n=== Duplicate Rows: {df.duplicated().sum()} ===')

Cek distribusi kelas dan visualisasinya

In [ ]:
print('\n=== Class Distribution ===')
print(df['Kategori'].value_counts())

# Visualisasi
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Kategori', palette='viridis')
plt.title('Distribution of Spam vs Ham Messages', fontsize=14, fontweight='bold')
plt.xlabel('Category', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
for i, v in enumerate(df['Kategori'].value_counts()):
    plt.text(i, v + 50, str(v), ha='center', va='bottom', fontsize=11)
plt.tight_layout()
plt.show()



Cek analysis seperti panjangnya messange, melakukan visualisasi dan word count

In [ ]:
# Analisa panjangnya pesan
df['message_length'] = df['Pesan'].apply(lambda x: len(str(x)))
df['word_count'] = df['Pesan'].apply(lambda x: len(str(x).split()))

print('\n=== Message Length Statistics ===')
print(df.groupby('Kategori')[['message_length', 'word_count']].describe())

# Visualisasi dari distribusi panjangnya pesan
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Visualisasi dari panjangnya pesan
df.boxplot(column='message_length', by='Kategori', ax=axes[0])
axes[0].set_title('Message Length by Category')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Character Count')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Visualisasi Word count
df.boxplot(column='word_count', by='Kategori', ax=axes[1])
axes[1].set_title('Word Count by Category')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Word Count')
plt.sca(axes[1])
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

# Sample messages
print('\n=== Sample Spam Messages ===')
print(df[df['Kategori'] == 'Spam']['Pesan'].head(3).values)
print('\n=== Sample Ham Messages ===')
print(df[df['Kategori'] == 'ham']['Pesan'].head(3).values)

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning.

Jika Anda menggunakan data teks, data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah. Khususnya ketika kami menggunakan data tidak terstruktur.

1. Menangani Data Kosong

In [ ]:
# 2. Handle missing values
df = df.dropna()
print(f'After removing missing values: {len(df)} rows')

2. Menghapus Data Duplikat

In [ ]:
print(f'Before removing duplicates: {len(df)} rows')
df = df.drop_duplicates()
print(f'After removing duplicates: {len(df)} rows')

3. Dilakukan data encoding (kategorical) dan standarisasi fitur

In [ ]:
def clean_text(text):
    # Convert ke lowercase
    text = str(text).lower()
    
    # hapus urls
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # hapus email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # hapus phone numbers
    text = re.sub(r'\d{5,}', '', text)
    
    # hapus special characters dan digits
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # hapus whitespace  
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# apply text cleaning
print('\nCleaning text...')
df['cleaned_text'] = df['Pesan'].apply(clean_text)

# sesudah dan sebelum text cleaning
print('\n=== Text Cleaning Examples ===')
for i in range(3):
    print(f'\nOriginal: {df.iloc[i]["Pesan"]}')
    print(f'Cleaned: {df.iloc[i]["cleaned_text"]}')

4. Encode Target Variable   

In [ ]:

df['label'] = df['Kategori'].map({'Spam': 1, 'ham': 0})

5. Data Binning

Split data ke training dan testing

In [ ]:
X = df['cleaned_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'\n=== Train-Test Split ===')
print(f'Training set: {len(X_train)} samples')
print(f'Testing set: {len(X_test)} samples')


Vektorisasi

In [ ]:
print('\nPerforming TF-IDF vectorization...')
vectorizer = TfidfVectorizer(max_features=3000, min_df=2, max_df=0.8)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f'TF-IDF matrix shape (train): {X_train_tfidf.shape}')
print(f'TF-IDF matrix shape (test): {X_test_tfidf.shape}')


Simpan data yang sudah dilakukan preprocessing

In [ ]:
preprocessed_df = df[['Kategori', 'Pesan', 'cleaned_text', 'label']].copy()
preprocessed_df.to_csv('preprocessing/indo_spam_preprocessing.csv', index=False)
print('\nPreprocessed data saved to: preprocessing/indo_spam_preprocessing.csv')

6. Validasi model untuk training cepat, prediksi, evaluasi serta Confusion Matrix

In [ ]:
print('\n=== Training a Quick Naive Bayes Model for Validation ===')
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Prediksi
y_pred = nb_model.predict(X_test_tfidf)

# Evaluasi
print(f'\nAccuracy: {accuracy_score(y_test, y_pred):.4f}')
print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix - Naive Bayes Classifier', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

print('\n=== Data Preprocessing Completed! ===')